In [154]:
#run this everytime an edit is made in utils.py so that we can use the helper functions in this Python Notebook
import importlib
import utils_hw2
importlib.reload(utils_hw2)

<module 'utils_hw2' from '/Users/ivychan/finm-375-fixed-income-deriv-a/ivy-work/homework-2/utils_hw2.py'>

# Exercise - Pricing Swaptions


# 1. Pricing the Swaption


## Swaption Vol Data

The file `data/swaption_vol_data_2025-06-30.xlsx` has market data on the implied volatility skews for swaptions. Note that it has several columns:
* `expry`: expiration of the swaption
* `tenor`: tenor of the underlying swap
* `model`: the model by which the volatility is quoted. (All are Black.)
* `-200`, `-100`, etc.: The strike listed as difference from ATM strike (bps). Note that ATM is considered to be the **forward swapa rate** which you can calculate.


Your data: you will use a single row of this data for the `1x4` swaption.
* date: `2025-06-30`
* expiration: 1yr
* tenor: 4yrs


In [155]:
#reading the data from Excel into a pandas Dataframe
import pandas as pd
import numpy as np

vol_path = "data-hw2/swaption_vol_data_2025-06-30.xlsx"
vol_data = pd.read_excel(vol_path)

row1x4vols = vol_data.loc[vol_data["tenor"] == 4]
display(row1x4vols)

row1x4vols = row1x4vols.iloc[0] #make into scalar for easier calculations

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
3,SOFR,swaption,black,2025-06-30,1,4,54.405,38.565,33.925,32.195,30.83,29.805,29.095,28.43,28.885


## Rate Data

The file `data/cap_curves_2025-06-30.xlsx` gives 
* SOFR swap rates, 
* their associated discount factors
* their associated forward interest rates.

You will not need the cap data (flat or forward vols) for this problem.


In [156]:
tenor_path = "data-hw2/cap_curves_2025-06-30.xlsx"
tenor_data = pd.read_excel(tenor_path)

tenor_subset = tenor_data[["tenor", "swap rates", "spot rates", "discounts", "forwards"]].copy()
tenor_subset.head()

,tenor,swap rates,spot rates,discounts,forwards
0,0.25,0.042353,0.042353,0.989523,NaN
1,0.50,0.040859,0.040852,0.979883,0.039351
2,0.75,0.039391,0.039372,0.971043,0.036414
3,1.00,0.038115,0.038083,0.962807,0.034217
4,1.25,0.036704,0.036653,0.955417,0.030938


## The Swaption

Consider the following swaption with the following features:
* underlying is a fixed-for-floating (SOFR) swap
* the underlying swap has **quarterly** payment frequency
* this is a **payer** swaption, which gives the holder the option to **pay** the fixed swap rate and receive SOFR.


### 1.1
Calculate the (relevant) forward swap rate. That is, the one-year forward 4-year swap rate.


In [157]:
SWAP_START = 1.0
SWAP_TENOR = 4.0
FREQUENCY = 0.25
NOTIONAL = 100.0

discounts_df = tenor_subset[["tenor","discounts"]]

forward_rate_1x4 = utils_hw2.calc_forward_swap_rate(discounts_df, SWAP_START, SWAP_TENOR, FREQUENCY)

print(f"1x4 Forward Swap Rate = {forward_rate_1x4:.6f}")
print(f"In percent = {100 * forward_rate_1x4:.4f}%")

1x4 Forward Swap Rate = 0.032698
In percent = 3.2698%


### 1.2
Price the swaptions at the quoted implied volatilites and corresponding strikes, all using the just-calculated forward swap rate as the underlying.

In [158]:
strike_offset_bps = [-200, -100, -50, -25, 0, 25, 50, 100, 200]
annuity = utils_hw2.calc_swap_annuity(discounts_df, SWAP_START, SWAP_TENOR, FREQUENCY)

results = []
for offset_bps in strike_offset_bps: 
    black_vol = row1x4vols[offset_bps] / 100.0 #convert from percent to decimal
    strike = forward_rate_1x4 + offset_bps / 10000.0 #strike = ATM + shift

    price = utils_hw2.black_price_payer_swaption(black_vol, SWAP_START, strike, forward_rate_1x4, annuity, NOTIONAL)

    results.append({
        "offset bps": offset_bps,
        "strike": round(strike * 100, 4), #convert to percent to match the given data
        "impl vol": round(black_vol * 100, 4), #convert to percent to match the given data
        "price": round(price, 6)
    })

prices_df = pd.DataFrame(results).sort_values("offset bps").reset_index(drop = True).set_index("offset bps")
prices_df

,strike,impl vol,price
offset bps,,,
-200,1.2698,54.405,7.271096
-100,2.2698,38.565,3.948049
-50,2.7698,33.925,2.536235
-25,3.0198,32.195,1.943130
0,3.2698,30.830,1.443366
25,3.5198,29.805,1.042391
50,3.7698,29.095,0.736560
100,4.2698,28.430,0.355738
200,5.2698,28.885,0.087983


### 1.3
To consider how the expiration and tenor matter, calculate the prices of a few other swaptions for comparison. 
* No need to get other implied vol quotes--just use the ATM implied vol you have for the swaption above. (Here we are just interested in how Black's formula changes with changes in tenor and expiration.)
* No need to calculate for all the strikes--just do the ATM strike.

Alternate swaptions
* The 3mo x 4yr swaption
* The 2yr x 4yr swaption
* the 1yr x 2yr swaption

Report these values and compare them to the price of the `1y x 4y` swaption.

In [160]:
ATM_BLACK_VOL = row1x4vols[0] / 100.0 #convert the ATM vol (offset 0) from percent to decimal

alt_swaptions = [
    {"label": "1y x 4y", "expiry": SWAP_START, "swap_start": SWAP_START, "swap_tenor": SWAP_TENOR}, #already calculated in question 1.2
    {"label": "3m x 4y", "expiry": 0.25, "swap_start": 0.25, "swap_tenor": 4.0},
    {"label": "2y x 4y", "expiry": 2.00, "swap_start": 2.00, "swap_tenor": 4.0},
    {"label": "1y x 2y", "expiry": 1.00, "swap_start": 1.00, "swap_tenor": 2.0},
]

results = []

for swaption in alt_swaptions:
    expiry = swaption["expiry"]
    swap_start = swaption["swap_start"]
    swap_tenor = swaption["swap_tenor"]

    forward_swap_rate = utils_hw2.calc_forward_swap_rate(discounts_df, swap_start, swap_tenor, FREQUENCY)
    annuity = utils_hw2.calc_swap_annuity(discounts_df, swap_start, swap_tenor, FREQUENCY)

    strike = forward_swap_rate

    price = utils_hw2.black_price_payer_swaption(ATM_BLACK_VOL, expiry, strike, forward_swap_rate, annuity, NOTIONAL)

    results.append({
        "swaption": swaption["label"],
        "expiry years": expiry,
        "swap tenor years": swap_tenor,
        "forward swap rate": round(forward_swap_rate, 6),
        "strike": round(strike * 100, 6), #convert to percent to match the given data
        "annuity": round(annuity, 6),
        "impl vol": ATM_BLACK_VOL * 100, #convert to percent to match the given data
        "price": round(price, 6)   
    })

compare_df = pd.DataFrame(results)
compare_df

,swaption,expiry years,swap tenor years,forward swap rate,strike,annuity,impl vol,price
0,1y x 4y,1.00,4.0,0.032698,3.269770,3.603238,30.83,1.443366
1,3m x 4y,0.25,4.0,0.032998,3.299759,3.692194,30.83,0.748498
2,2y x 4y,2.00,4.0,0.034257,3.425671,3.484495,30.83,2.059942
3,1y x 2y,1.00,2.0,0.031183,3.118344,1.860504,30.83,0.710757


Recall that Black's Formula for swaptions is: $B_{\text{call}}(t) = Z_{\text{swap}}(0,T_0,T) \left[F_t N(d_1) - K N(d_2)\right]$ with $d_1 =\frac{\ln\!\left(\frac{F_t}{K}\right)+\frac{1}{2}\sigma^2 \tau} {\sigma \sqrt{\tau}}$ and $d_2 = d_1 - \sigma \sqrt{\tau}$

Since we are pricing ATM, we have $K = F_t$ which eliminates the log-moneyness term of Black's Formula for swaptions. Therefore, $d_1 = \frac{1}{2}\sigma\sqrt{\tau}$ and $d_2 = -\frac{1}{2}\sigma\sqrt{\tau}$.

This further simplifies the price to: $B_{\text{ATM}} = Z_{\text{swap}}(0,T_0,T) \cdot F_t \left[2N\!\left(\frac{1}{2}\sigma\sqrt{\tau}\right)-1\right]$


Based on this, it is evident that the ATM swaption value is driven by:
1. $Z_{\text{swap}}(0,T_0,T) = \sum_{i=1}^{m} Z(0,T_i)$ or the annuity
2. $F_t$ or the level of the forward swap rate
3. $\left[2N\!\left(\frac{1}{2}\sigma\sqrt{\tau}\right)-1\right]$ or the "optionality term" which increases with both volatility $\sigma$ and expiration $\tau$. 

<br>

**Expiration**

Based on the decomposition above, it's clear that expiration enters through the optionality term $2N\!\left(\tfrac{1}{2}\sigma\sqrt{\tau}\right)-1$. 

Because $\sqrt{\tau}$ increases with time to expiration, a longer-dated option increases $d_1$ in magnitude and therefore increases the spread between $N(d_1)$ and $N(d_2)$. Economically, a longer expiration gives more time for the forward swap rate $F_t$ to move away from the strike, increasing the probability of ending in-the-money. This explains why the 2y × 4y swaption is more valuable than the 1y × 4y swaption in the results, even though both reference the same underlying 4-year swap.

<br>

**Tenor**

It's also evident that the swap tenor affects the annuity term $Z_{\text{swap}}(0,T_0,T) = \sum_{i=1}^{m} Z(0,T_i)$. A longer swap tenor increases the number of fixed-leg payment dates and therefore increases the sum of discount factors. Since the swaption price is proportional to this annuity, a longer underlying swap mechanically increases the dollar value of the option. This is apparent in the output above (we're holding expiration constant): the 1y × 4y swaption is substantially more valuable than a 1y × 2y swaption and the 4-year swap has nearly double the annuity of the 2-year swap.

<br>

Overall, the expiration does not change the annuity term -- it only changes the optionality component. In contrast, tenor changes the annuity but leaves the optionality term (for fixed $\sigma$ and $\tau$) unchanged.
The pricing formula therefore cleanly separates time-to-expiry risk (optionality) from swap-length exposure (annuity) which is what we observe numerically in the output above.